In [ ]:
import os, gzip, subprocess
import numpy as np

from nilearn.image import smooth_img
from nilearn.maskers import NiftiLabelsMasker, NiftiMasker
from nilearn.interfaces import fmriprep

space = 'MNI152NL'
desc = 'preproc_bold.nii.gz'
#-------------------
preproc_data = './cneuromod.mutemusic.fmriprep'
mistroicsv = './parcellation_data/MIST_ROI.csv'
mistroi_labelsimg = './parcellation_data/MIST_ROI.nii.gz'
parcel_dir = './parcellation_data/'

In [ ]:
def correct_partmag_pos(filepath, destdir):
    basedir, basename = os.path.split(filepath)
    basename = basename.replace('_part-mag', '')
    #i = basename.find('_run-')+len('_run-1')
    #corrected_basename = basename[:i]+'_part-mag'+basename[i:]
    return os.path.join(destdir, basename)

def wholebrain_voxel(fmri_path, fake_fmri_path, mask_img, savepath, tr=1.49, smoothing_fwhm=8):
    mymasker = NiftiMasker(mask_img=mask_img,
                                 standardize="zscore_sample",detrend=False,t_r=tr,smoothing_fwhm=smoothing_fwhm)
    mymasker.fit()
    confounds, sample_mask = fmriprep.load_confounds(fake_fmri_path,
                                       strategy=['high_pass', 'motion', 'wm_csf', 'global_signal'],
                                       motion="basic", wm_csf="basic", global_signal="basic")
    X = mymasker.fit_transform(fmri_path,confounds=confounds, sample_mask=sample_mask)
    print(f'sample mask is {sample_mask}, saving wholebrain voxels {X.shape}...')
    np.savez_compressed(savepath,X=X)

def parcellation(fmri_path, fake_fmri_path, mask_img, labels_img, savepath, tr=1.49, smoothing_fwhm=8):
    mymasker = NiftiLabelsMasker(labels_img=labels_img, mask_img=mask_img,
                                 standardize="zscore_sample",detrend=False,t_r=tr,smoothing_fwhm=smoothing_fwhm)
    mymasker.fit()
    confounds, sample_mask = fmriprep.load_confounds(fake_fmri_path,
                                       strategy=['high_pass', 'motion', 'wm_csf', 'global_signal'],
                                       motion="basic", wm_csf="basic", global_signal="basic")
    X = mymasker.fit_transform(fmri_path,confounds=confounds, sample_mask=sample_mask)
    print('saving ROI parcellations ...')
    np.savez_compressed(savepath,X=X)

def create_temp_conf(conf_path, destdir):
    temp_conf = correct_partmag_pos(conf_path, destdir)
    subprocess.run(
        f"cp {conf_path} {temp_conf}", shell=True, executable='/bin/bash'
    )
    return temp_conf

def remove_temp_conf(conf_path):
    subprocess.run(
        f"rm -f {conf_path}", shell=True, executable='/bin/bash'
    )

In [ ]:
data_path = 'cneuromod.mutemusic.fmriprep'
subject = 'sub-03'
sub_path = os.path.join(data_path, subject)

space = 'space-MNI152NLin2009cAsym'
desc = 'desc-preproc'

dest = os.path.join(parcel_dir, 'WB_voxels', subject)
os.makedirs(dest, exist_ok=True)

sessions = [os.path.join(sub_path, dir, 'func') for dir in os.listdir(sub_path) if 'ses' in dir]
for ses_path in sessions:
    files = os.listdir(ses_path)
    bolds = [file for file in files if (space in file)and('bold.nii.gz' in file)]
    for bold_file in bolds:
        bold_path = os.path.join(ses_path, bold_file)
        mask_path = bold_path.replace('desc-preproc_part-mag_bold.nii.gz', 
                                      'desc-brain_part-mag_mask.nii.gz')
        conf_path = bold_path.replace('space-MNI152NLin2009cAsym_desc-preproc_part-mag_bold.nii.gz', 
                                      'desc-confounds_part-mag_timeseries.tsv')
        fake_bold_path = correct_partmag_pos(bold_path, dest)
        fake_conf = create_temp_conf(conf_path, destdir=dest)

        output_path = fake_bold_path.replace('_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz','')
        print(output_path)
        
        parcellation(fmri_path=bold_path, fake_fmri_path=fake_bold_path,
                    mask_img=mask_path, labels_img=mistroi_labelsimg,
                    savepath=output_path)
        
        remove_temp_conf(fake_conf)